In [476]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn import linear_model


In [477]:
import os
import re

import nltk

alice_file = 'alice.txt'
alice_raw = None

if not os.path.isfile(alice_file):
    from urllib import request
    url = 'http://www.gutenberg.org/cache/epub/19033/pg19033.txt'
    response = request.urlopen(url)
    alice_raw = response.read().decode('utf8')
    with open(alice_file, 'w', encoding='utf8') as f:
        f.write(alice_raw)
else:
    with open(alice_file, 'r', encoding='utf8') as f:
        alice_raw = f.read()

In [478]:
# For reasons, lets remove the start and end bloat from the text
start = "I--DOWN THE RABBIT-HOLE"
end = "End of the Project Gutenberg"
start_index = alice_raw.find(start)
end_index = alice_raw.rfind(end)
alice = alice_raw[start_index:end_index]

# And replace more than one subsequent whitespace chars with one space
alice = re.sub(r'\s+', ' ', alice)

In [479]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
     
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()
     
# Use tokenize method
tokens = tk.tokenize(alice)
     
train_data = tokens[:8691]
test_data = tokens[8691:]

In [553]:
tokens_y = []
for word in tokens:
    y = ""
    for i in range (len(word)-1):
        y = y+'0'
    y = y+'1'
    tokens_y.append(y)

In [554]:
x = ''.join(tokens)
y = ''.join(tokens_y)

In [574]:
features = []
x_input = []
output = []
for i in range(len(x)):
    x_input.append(x[i])
    left = 0
    if i-5 > 0:
        left = i-5
    right = len(x)-1
    if i+5 < len(x):
        right = i+5
    xi = []
    for j in range(left, right+1):
        xi.append(x[j])
    features.append(xi)
    output.append(y[i])

In [588]:
df = pd.DataFrame(features)
df.columns =['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']
df['x'] = x_input
df['y'] = output

In [624]:
df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,x,y
0,I,-,-,D,O,W,None,None,None,None,None,I,0
1,I,-,-,D,O,W,N,None,None,None,None,-,0
2,I,-,-,D,O,W,N,T,None,None,None,-,0
3,I,-,-,D,O,W,N,T,H,None,None,D,0
4,I,-,-,D,O,W,N,T,H,E,None,O,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42509,u,s,t,r,a,t,i,o,n,],None,t,0
42510,s,t,r,a,t,i,o,n,],None,None,i,0
42511,t,r,a,t,i,o,n,],None,None,None,o,0
42512,r,a,t,i,o,n,],None,None,None,None,n,0


# Feaature Selction using one hot encoding

In [597]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
onehot = encoder.fit_transform(df.loc[:,'f1':'f11'])

In [618]:
df_os = pd.DataFrame(onehot)
df_os.columns = [str(i) for i in range(onehot.shape[1])]

In [619]:
df.os

,0,1,2,3,4,5,6,7,8,9,...,732,733,734,735,736,737,738,739,740,741
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [620]:
df_combined = pd.concat([df, df_os], axis=1, join='inner')
df_combined

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,732,733,734,735,736,737,738,739,740,741
0,I,-,-,D,O,W,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,I,-,-,D,O,W,N,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,I,-,-,D,O,W,N,T,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,I,-,-,D,O,W,N,T,H,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,I,-,-,D,O,W,N,T,H,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42509,u,s,t,r,a,t,i,o,n,],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42510,s,t,r,a,t,i,o,n,],None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42511,t,r,a,t,i,o,n,],None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42512,r,a,t,i,o,n,],None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [622]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_combined, test_size=0.2, shuffle=False)

In [628]:
train.loc[:,'0':'741']

,0,1,2,3,4,5,6,7,8,9,...,732,733,734,735,736,737,738,739,740,741
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [629]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=1000).fit(train.loc[:,'0':'741'], train['y'])

In [630]:
clf.score(test.loc[:,'0':'741'], test['y'])

0.8961542984828884

In [649]:
y_pred = clf.predict(test.loc[:,'0':'741'])
for i in range(test.shape[0]):
    print(test['x'].iloc[i],end="")
    if(y_pred[i] == '1'):
        print(" ",end="")

n t ohe r face. "Very," said Alice. "Where's the Duchess?" "Hush! Hush!" said the Rabbit. " She's undersenten ce ofexe cution.""What for?" said Alice."She boxed the Queen's e ars--" the Rabbit began. "Gettoy ourplaces!" shouted the Queen ina voice of thunder, and people beg anrunning a bout inalld irections, tumbling upagains teachother. However, they got settled down inaminuteortwo, and the game began. Alice thought she had neverseensuchacurious croquet-ground inherlife; it was allrid ges and furrows. The croquet balls werelive hedgehogs, and the malletslive flamingos and the sold ier shad tod ouble themselvesupand stand on theirhand sand feet, to make the ar ches. The player sallplayed aton ce, wit hout waitin g forturns, quarrelling all the while and fightin g for thehedgehogs; and in avery shorttime, the Queen was in afur ious pas sion and went s tamping a bout and shouting, "Off wit h hishead!"or"Off wit h herhead!" abouton ce inaminute. "They 'redread fully fond of behead ing peo

er the irverdict.""No, no!" said the Queen. "Sentence first--verdict af ter wards.""Stuffand nonsense!" said Alice loudly."Theide ao f having thesentence first!""Hold yourtongue!" said the Queen, turning purple. "I won't!" said Alice. "Off wit h herhead!" the Queen shouted at the topofher voice. No body moved. "Who care s for_you_?" said Alice (she had grownt oherfull size by this time). "You're nothing but a packof cards!" [Illustration] At this, the whole packroseupint he air and camefly ing downuponher; she gave alitt lescream, halfoffright and half of anger, and tried to beat the moff, an d found herselfly ing on the bank, wit hherhead inthe lapofher sister, who wasgently brushing away somedead le aves thathad fluttered downfrom the treesuponhe r face. "Wakeup, Alice dear!" said her sister. "Why, what alongsleepyou'vehad!""Oh, I've had suchacuriousdream!" said Alice. And she told her sister, aswellas she could re member them, all the sestrangeadven ture sofhersthat y ouhave just be

In [651]:
actual = test['y']
predicted = y_pred

tp = np.sum((actual=='1') & (predicted=='1'))
fp = np.sum((actual!='1') & (predicted=='1'))
tn = np.sum((actual=='0') & (predicted=='0'))
fn = np.sum((actual!='0') & (predicted=='0'))

accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2 * (precision * recall) / (precision + recall)

In [652]:
print (accuracy, precision, recall, f1)

0.8961542984828884 0.8014571948998178 0.7036247334754797 0.7493613397672437
